# Workshop Handoff: Sistema de Decisiones de la USS Enterprise-D

## 🎯 ¿Qué es el Patrón Handoff?

El patrón **Handoff** es una estrategia de orquestación donde **un agente especializado evalúa la situación y decide dinámicamente a qué otro agente delegar la acción**.

A diferencia de Magentic (que itera y replantea), Handoff es una **toma de decisión singular y clara**: un agente decide, el siguiente ejecuta.

### 🚀 Comparativa: Magentic vs Handoff

| Característica | **Magentic** | **Handoff** |
|---|---|---|
| **Iteraciones** | Múltiples rondas (gestor replantea) | Una sola decisión y ejecución |
| **Decisor** | Gestor central | Agente especialista |
| **Complejidad** | Alta (para problemas complejos) | Media (decisiones claras) |
| **Velocidad** | Lenta (múltiples iteraciones) | Rápida (ejecución directa) |
| **Caso de uso** | Investigación, planificación | Aprobaciones, validaciones |

### 🎬 Ejemplo Star Trek: Situación de Emergencia

**Escenario**: Capitán Picard detecta una vela iónica en las cercanías.

```
┌──────────────────────────────┐
│  CAPITÁN PICARD (Evaluador)  │
│  ¿Es una amenaza?            │
│  ¿Riesgo: Alto/Medio/Bajo?   │
└──────────┬───────────────────┘
           │
      Analiza
           │
    ┌──────┴─────────┐
    │                │
 RIESGO ALTO      RIESGO BAJO
    │                │
    ▼                ▼
┌─────────┐    ┌──────────┐
│ WORF    │    │ RIKER    │
│Seguridad│    │Exploración
│Ejecuta  │    │Ejecuta
│Defensa  │    │Acercamiento
└─────────┘    └──────────┘
    │                │
    └────────┬───────┘
             │
    ┌────────▼─────────┐
    │ GEORDI           │
    │Oficial de Bitácora
    │Registra acción   │
    └──────────────────┘
```

### 💡 Componentes del Patrón Handoff

1. **Evaluador**: Analiza y clasifica la situación (Picard → "¿Riesgo?")
2. **Ejecutores**: Múltiples agentes especializados (Worf, Riker, etc.)
3. **Registro**: Bitácora que documenta la acción tomada
4. **Rutas condicionales**: El evaluador ruta a ejecutores según criterios

## Objetivos de Este Workshop
- ✅ Entender el patrón Handoff vs Magentic
- ✅ Construir un flujo decisional con HandoffBuilder
- ✅ Implementar evaluación + ejecución condicional
- ✅ Registrar decisiones en bitácora

> **Contexto**: Trabajarás en la USS Enterprise-D resolviendo incidentes con decisiones rápidas y delegación clara.

## 1) Configuración del entorno
Verifica que tus variables de entorno están disponibles (Azure OpenAI).

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

base_url = os.getenv("AZURE_OPENAI_ENDPOINT")
api_key = os.getenv("AZURE_OPENAI_API_KEY")
model_id = os.getenv("AZURE_OPENAI_DEPLOYMENT")

print("AZURE_OPENAI_ENDPOINT:", "OK" if base_url else "MISSING")
print("AZURE_OPENAI_API_KEY:", "OK" if api_key else "MISSING")
print("AZURE_OPENAI_DEPLOYMENT:", model_id if model_id else "MISSING")

AZURE_OPENAI_ENDPOINT: OK
AZURE_OPENAI_API_KEY: OK
AZURE_OPENAI_DEPLOYMENT: gpt-4


## 2) Concepto clave: Handoff

**Handoff** = el sistema decide qué agente actúa a continuación según el contexto.

En este taller:
- **Coordinador** decide el riesgo.
- **Ejecutor** ejecuta si procede.
- **Bitácora** registra el resultado.

El flujo se define con `HandoffBuilder` (MAF) y se ejecuta con `run_stream()`.

In [2]:
from agent_framework import ChatAgent, HandoffBuilder, WorkflowOutputEvent, AgentRunUpdateEvent
from agent_framework.openai import OpenAIChatClient

## 3) Ejercicio 1: Crear agentes

Crea los 3 roles básicos del flujo.

In [3]:
coordinador = ChatAgent(
    chat_client=OpenAIChatClient(
        base_url=base_url,
        api_key=api_key,
        model_id=model_id
    ),
    name="Coordinador",
    instructions="""Eres el Coordinador de la Flota Estelar.
SIEMPRE responde en este formato exacto:
RIESGO: [bajo/medio/alto]
JUSTIFICACIÓN: [explicación clara de 1-2 oraciones]

Ejemplo:
RIESGO: alto
JUSTIFICACIÓN: Desactivar escudos deja la nave vulnerable a ataques enemigos durante 30 minutos.

Responde en español. NO agregues nada más."""
)

ejecutor = ChatAgent(
    chat_client=OpenAIChatClient(
        base_url=base_url,
        api_key=api_key,
        model_id=model_id
    ),
    name="Ejecutor",
    instructions="""Eres el Ejecutor. Ejecuta operaciones aprobadas.
Responde breve en espanol."""
)

bitacora = ChatAgent(
    chat_client=OpenAIChatClient(
        base_url=base_url,
        api_key=api_key,
        model_id=model_id
    ),
    name="Bitacora",
    instructions="""Eres el Oficial de Bitacora.
Responde con: [ACCION]: [RESULTADO] en 1 oracion. Espanol."""
)

print("Agentes listos:", coordinador.name, ejecutor.name, bitacora.name)

Agentes listos: Coordinador Ejecutor Bitacora


## 4) Ejercicio 2: Construir el workflow HandoffBuilder

### 📋 Constructor HandoffBuilder Detallado

El `HandoffBuilder` es un **patrón fluido** (fluent pattern) que construye un workflow de decisiones en cadena. Veamos cada línea:

```python
workflow = (
    HandoffBuilder(participants=[coordinador, ejecutor, bitacora])  # Paso 1: Registrar participantes
    .set_coordinator(coordinador)                                    # Paso 2: Designar evaluador
    .add_handoff(coordinador, [ejecutor, bitacora])                # Paso 3: Rutas de salida
    .add_handoff(ejecutor, bitacora)                               # Paso 4: Segunda handoff
    .build()                                                         # Paso 5: Compilar workflow
)
```

#### 🔍 Explicación Línea por Línea

**Línea 1: `HandoffBuilder(participants=[coordinador, ejecutor, bitacora])`**
- **Qué hace**: Crea un constructor que registra los 3 agentes que participarán en el workflow
- **Parámetro**: `participants` = lista de agentes ChatAgent
- **Analogía Star Trek**: "Capitán Picard convoca a Worf, Riker y Geordi a la sala de conferencias"

**Línea 2: `.set_coordinator(coordinador)`**
- **Qué hace**: Designa quién será el **evaluador/decisor inicial**
- **El coordinador**: Es el agente que recibe la entrada y decide a dónde enviarla
- **En nuestro caso**: Picard es quien evalúa el riesgo (alto/medio/bajo)
- **Importante**: El coordinador SIEMPRE actúa primero, analizando la tarea

**Línea 3: `.add_handoff(coordinador, [ejecutor, bitacora])`**
- **Qué hace**: Define las **rutas de salida desde el coordinador**
- **Sintaxis**: `add_handoff(origen, destinos)`
  - `origen`: Agente que decide (Coordinador)
  - `destinos`: Lista de agentes a los que PUEDE delegar (Ejecutor O Bitácora)
- **Cómo elige**: El framework usa el análisis del coordinador para elegir cuál de los destinos
- **En nuestro caso**:
  ```
  Coordinador → ¿Riesgo BAJO? → Ejecutor
  Coordinador → ¿Riesgo ALTO? → Bitácora (registrar rechazo)
  ```

**Línea 4: `.add_handoff(ejecutor, bitacora)`**
- **Qué hace**: Define la **segunda transición**
- **Flujo**: Después que el Ejecutor actúa, siempre va a la Bitácora
- **Propósito**: Garantizar que cada acción queda registrada
- **En nuestro caso**:
  ```
  Ejecutor (ejecuta la acción aprobada) → Bitácora (registra en logs)
  ```

**Línea 5: `.build()`**
- **Qué hace**: Compila y valida el workflow
- **Retorna**: Un objeto `Workflow` listo para ejecutar con `.run_stream()`
- **Validaciones internas**: Verifica que no haya ciclos infinitos, que todos los destinos sean válidos, etc.

---

#### 📊 Mapa Visual del Flujo Handoff

```
ENTRADA: "¿Es seguro desactivar escudos?"
│
▼
┌─────────────────────────────────────────────┐
│  COORDINADOR (Picard)                       │
│  Analiza y determina RIESGO                 │
│  Output: "RIESGO: alto" o "RIESGO: bajo"    │
└────────┬────────────────────────┬───────────┘
         │                        │
    RIESGO BAJO              RIESGO ALTO
         │                        │
         ▼                        │
    ┌────────────┐                │
    │  EJECUTOR  │                │
    │ Ejecuta    │                │
    │ acción     │                │
    │ aprobada   │                │
    └────┬───────┘                │
         │                        │
         │   Segunda Handoff:     │
         │   Ejecutor → Bitácora  │
         │                        │
         └────────────┬───────────┘
                      │
                      ▼
         ┌──────────────────────────┐
         │  BITÁCORA (Geordi)       │
         │  Registra resultado      │
         │  final en logs           │
         │                          │
         │  - Si viene de Ejecutor: │
         │    "Acción completada"   │
         │  - Si viene de Coord:    │
         │    "Operación rechazada" │
         └──────────────────────────┘


RESUMEN DE FLUJOS:
═════════════════════════════════════════════

Flujo 1 - RIESGO BAJO (Ejecutar):
  Coordinador ──→ Ejecutor ──→ Bitácora
                               (Registra éxito)

Flujo 2 - RIESGO ALTO (Rechazar):
  Coordinador ──→ Bitácora
                 (Registra rechazo directo)
```

---

#### 🔧 Parámetros Clave

| Parámetro | Tipo | Ejemplo | Rol |
|-----------|------|---------|-----|
| `participants` | `list[ChatAgent]` | `[coordinador, ejecutor, bitacora]` | Todos los agentes del workflow |
| `coordinator` | `ChatAgent` | `coordinador` | Agente evaluador que actúa primero |
| `origen` (add_handoff) | `ChatAgent` | `coordinador` | De quién parte la decisión |
| `destinos` (add_handoff) | `list[ChatAgent]` | `[ejecutor, bitacora]` | A quién puede ir (el framework elige) |

---

#### ⚡ Diferencias vs Otros Patrones

| Builder | Flujo | Decisión |
|---------|-------|----------|
| **Sequential** | A → B → C (fijo) | Ninguna (orden predefinido) |
| **Concurrent** | A ∥ B ∥ C (paralelo) | Ninguna (todos actúan) |
| **Handoff** | A → (B O C) → D | **Un agente elige** |
| **Magentic** | A ↔ (B/C/D) múltiples veces | **Un gestor replantea iterativamente** |

---

Define las rutas: Coordinador → (Ejecutor o Bitácora) y Ejecutor → Bitácora.

In [4]:
workflow = (
    HandoffBuilder(participants=[coordinador, ejecutor, bitacora])
    .set_coordinator(coordinador)
    .add_handoff(coordinador, [ejecutor, bitacora])
    .add_handoff(ejecutor, bitacora)
    .build()
)

print("Workflow creado con HandoffBuilder")

Workflow creado con HandoffBuilder


## 5) Ejercicio 3: Ejecutar un escenario

Mira cómo se emiten eventos y cómo llega el resultado agregado.

In [5]:
import asyncio

async def ejecutar_un_escenario():
    entrada = "¿Es seguro desactivar los escudos defensivos durante 30 minutos para reparar el generador de energia?"
    print("Entrada:", entrada)
    print("-" * 60)
    print("Eventos:\n")

    event_map = {
        'WorkflowStartedEvent': '[*] Workflow iniciado',
        'RequestInfoEvent': '[?] Solicitando entrada',
        'HandoffEvent': '[->] Transicion (handoff)',
        'AgentRunUpdateEvent': '[!] Agente procesando',
        'WorkflowOutputEvent': '[OK] Workflow completado'
    }

    contador = 0
    agentes_respuestas = {}

    try:
        async for event in workflow.run_stream(entrada):
            event_type = type(event).__name__
            desc = event_map.get(event_type, '[?] ' + event_type)
            
            if event_type != 'AgentRunUpdateEvent':  # No mostrar todos los update events
                contador += 1
                print(f"[{contador:2d}] {desc}")
            
            # Capturar respuestas de agentes - sin interrumpir el flujo
            if isinstance(event, AgentRunUpdateEvent):
                executor_name = getattr(event, 'executor_id', None)
                update_data = getattr(event, 'data', None)
                
                if executor_name and update_data:
                    if executor_name not in agentes_respuestas:
                        agentes_respuestas[executor_name] = ""
                    
                    texto = getattr(update_data, 'text', '')
                    if texto:
                        agentes_respuestas[executor_name] += texto

    except Exception as e:
        print(f"[ERROR] {str(e)}")

    print("\n" + "-" * 60)
    print("\nRESULTADO FINAL:\n")
    
    if agentes_respuestas:
        for i, (agente, respuesta) in enumerate(agentes_respuestas.items(), 1):
            respuesta_limpia = respuesta.strip()
            if respuesta_limpia:
                print(f"[{i}] {agente}:")
                print(f"    {respuesta_limpia}\n")
    else:
        print("[!] No hay respuestas de agentes\n")

await ejecutar_un_escenario()

Entrada: ¿Es seguro desactivar los escudos defensivos durante 30 minutos para reparar el generador de energia?
------------------------------------------------------------
Eventos:

[ 1] [*] Workflow iniciado
[ 2] [?] WorkflowStatusEvent
[ 3] [?] ExecutorInvokedEvent
[ 4] [?] ExecutorCompletedEvent
[ 5] [?] SuperStepStartedEvent
[ 6] [?] ExecutorInvokedEvent
[ 7] [?] ExecutorCompletedEvent
[ 8] [?] SuperStepCompletedEvent
[ 9] [?] SuperStepStartedEvent
[10] [?] ExecutorInvokedEvent
[11] [?] ExecutorCompletedEvent
[12] [?] SuperStepCompletedEvent
[13] [?] SuperStepStartedEvent
[14] [?] ExecutorInvokedEvent
[15] [?] Solicitando entrada
[16] [?] WorkflowStatusEvent
[17] [?] ExecutorCompletedEvent
[18] [?] SuperStepCompletedEvent
[19] [?] WorkflowStatusEvent

------------------------------------------------------------

RESULTADO FINAL:

[1] Coordinador:
    RIESGO: alto
JUSTIFICACIÓN: Desactivar los escudos defensivos expone la nave a posibles ataques o eventos imprevistos mientras se rea

## 6) Cómo interpretar la salida

- **Eventos**: indican los pasos internos del workflow.
- **HandoffEvent**: transición entre agentes.
- **WorkflowOutputEvent**: salida final agregada.

> Si ves `RequestInfoEvent`, significa que el flujo espera más input del usuario.

---

## 🎓 Conclusiones y Próximos Pasos

### ✅ Lo que Aprendiste

| Concepto | Descripción |
|----------|-------------|
| **Patrón Handoff** | Un agente evaluador decide dinámicamente a qué otro agente delegar |
| **Evaluador Especializado** | El coordinador analiza una situación y clasifica el riesgo |
| **Rutas Condicionales** | Diferentes caminos según la decisión del evaluador (bajo/alto/medio) |
| **Delegación Clara** | Cada agente tiene un rol específico: evaluar, ejecutar, registrar |
| **Ejecución Rápida** | Una decisión → una ejecución (no hay iteraciones ni replanteos) |

### 🔍 Comparativa: Cuándo usar cada Patrón

```
┌──────────────────────────────────────────────────────────────┐
│                     MATRIZ DE SELECCIÓN                      │
├──────────────────────────────────────────────────────────────┤
│                                                               │
│ SEQUENTIAL:  Tareas lineales, paso a paso predefinido         │
│ Ej: A → B → C (siempre en ese orden)                         │
│                                                               │
│ CONCURRENT:  Perspectivas paralelas del mismo problema       │
│ Ej: A ∥ B ∥ C (todos analizan simultáneamente)              │
│                                                               │
│ HANDOFF:     Decisión única y clara, ejecución rápida        │
│ Ej: A decide → B O C ejecuta (una sola ruta)                │
│                                                               │
│ MAGENTIC:    Problemas complejos, iteración y replanificación│
│ Ej: A itera ↔ B/C/D (múltiples rondas adaptativas)          │
│                                                               │
└──────────────────────────────────────────────────────────────┘
```

### 🎯 Caso de Uso Real: Enterprise-D Decision Flow

En este workshop implementamos un flujo de aprobación de operaciones:

1. **Capitán Picard (Evaluador)**: Recibe solicitud y determina riesgo
2. **Will Riker (Ejecutor)**: Si riesgo es bajo, ejecuta la acción
3. **Geordi (Bitácora)**: Registra siempre el resultado final

**Ventaja**: Decisiones rápidas en situaciones de emergencia sin iteraciones innecesarias.

### 💡 Patrones Avanzados con Handoff

- ✅ **Multi-nivel**: Handoff → Handoff → Handoff (cadena de decisiones)
- ✅ **Criterios complejos**: El evaluador puede usar @ai_function para análisis profundo
- ✅ **Auditoría integrada**: Bitácora registra cada decisión y acción
- ✅ **Rollback**: Ejecutor puede reportar fallo y volver al evaluador
- ✅ **Integración con herramientas**: Ejecutor puede llamar a @ai_function decorators

### 🚀 Próximos Pasos Recomendados

1. **Experimenta con criterios**: Modifica cómo Picard clasifica el riesgo
2. **Añade herramientas**: Integra @ai_function al Ejecutor para acciones reales
3. **Crea cadenas**: Handoff después de Handoff para procesos multi-etapa
4. **Combina patrones**: Usa Handoff para evaluación + Magentic para ejecución compleja
5. **Monitorea decisiones**: Usa logs de Bitácora para auditar todas las acciones

### 🖖 Reflexión Final

El patrón **Handoff** es perfecto cuando necesitas:
- ✅ **Claridad de decisión**: Un agente especialista elige
- ✅ **Velocidad**: Una sola ruta, sin iteraciones
- ✅ **Responsabilidad**: Cada agente tiene un rol claro
- ✅ **Trazabilidad**: Registro de quién decidió y quién ejecutó

En la USS Enterprise-D, Picard confía en Riker y Worf para ejecutar decisiones rápidas.
En tu aplicación, Handoff garantiza que cada decisión es clara, documentada y ejecutada por el especialista correcto.

---

**¡Misión completada, oficial!** Has aprendido a orquestar flujos de decisión con Handoff.
Ahora eres capaz de implementar sistemas de aprobación y ejecución que son rápidos, claros y auditables. 🖖
